In [1]:
from plotnine.data import mtcars, mpg
from statsmodels.regression import linear_model
from statsmodels.formula import api as sm
from sklearn.linear_model import LinearRegression, LogisticRegression
import math
import pandas as pd
from io import StringIO
from statsmodels.stats.weightstats import ttest_ind, ztest
from statsmodels.stats.proportion import proportions_chisquare
from scipy.stats import linregress

In [2]:
mtcars

,name,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
7,Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
8,Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
9,Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


# Linear regression (part 1)

Explanation of some observations:
- P>|t|: The p-value of the t-test. Shows the level of statistical significance at 5% level. If < 0.05, you reject the null hypothesis (that the intercept is zero) and can conclude that the actual intercept is greater than zero.
- P>|z|: The p-value of the z-test
- t-test vs z-test:
  - both are used to accomplish the same thing
  - use the t-test when the sample size is <=30 and the z-test when the sample size is >30
  - z-test distribution is the normal distribution
  - t-test distribution is wider than the normal distribution because it needs to account for extra uncertainity
  - for larger samples, the difference between the two is insignificant but the t-test is better as it'll cover both cases
- R-squared: If 100%, it means that the model explains 100% of the variance of the dependent variable.
- Adjusted R-squared: A more reliable indicator that penalises the score for adding irrrelevant variables.
- F-statistic: measures the overall significance of the model, that none of the coefficients are zero.
- Df: Number of variables used in determining a value. For a mean, it would be the number of samples. For a model, it is the number of variables.
- 0.025 - 0.975: 97.5% confidence that the actual coefficient is not zero.
- Endogenous variables: the dependent variables.
- Exogenous variables: the independent variables.

In [3]:
lm_formula = "mpg ~ disp + cyl + hp"

In [4]:
lm1 = sm.ols(formula=lm_formula, data=mtcars).fit()
lm1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     30.88
Date:                Wed, 09 Oct 2024   Prob (F-statistic):           5.05e-09
Time:                        21:02:44   Log-Likelihood:                -79.009
No. Observations:                  32   AIC:                             166.0
Df Residuals:                      28   BIC:                             171.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     34.1849      2.591     13.195      0.000      28.878      39.492
disp          -0.0188      0.010     -1.811      0.081      -0.040       0.002
cyl           -1.2274      0.797     -1.540      0.135      -2.861       0.406
hp            -0.0147      0.015     -1.002      0.325      -0.045       0.015
==============================================================================
Omnibus:                        2.942   Durbin-Watson:                   1.606
Prob(Omnibus):                  0.230   Jarque-Bera (JB):                2.558
Skew:                           0.675   Prob(JB):                        0.278
Kurtosis:                       2.692   Cond. No.                     1.51e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Linear regression (part 2) [WARNING]

This one produces wrong results. One should use dmatrices()

In [5]:
X = mtcars[["disp", "cyl", "hp"]]
y = mtcars["mpg"]

lm2 = linear_model.OLS(exog=X, endog=y).fit()
lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    mpg   R-squared (uncentered):                   0.866
Model:                            OLS   Adj. R-squared (uncentered):              0.852
Method:                 Least Squares   F-statistic:                              62.29
Date:                Wed, 09 Oct 2024   Prob (F-statistic):                    9.43e-13
Time:                        21:02:44   Log-Likelihood:                         -110.63
No. Observations:                  32   AIC:                                      227.3
Df Residuals:                      29   BIC:                                      231.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
disp          -0.0985      0.022     -4.406      0.000      -0.144      -0.053
cyl            7.8695      1.057      7.445      0.000       5.708      10.031
hp            -0.0501      0.038     -1.319      0.198      -0.128       0.028
==============================================================================
Omnibus:                        6.386   Durbin-Watson:                   0.828
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                2.047
Skew:                          -0.120   Prob(JB):                        0.359
Kurtosis:                       1.784   Cond. No.                         226.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Linear regression (part 3)

In [6]:
lm3 = LinearRegression().fit(X=X, y=y)

display(lm3.coef_)
display(lm3.feature_names_in_)
display(lm3.intercept_)

array([-0.01883809, -1.22741994, -0.01467933])

array(['disp', 'cyl', 'hp'], dtype=object)

34.18491916752096

# Linear regression (part 4)

In [7]:
from patsy import dmatrices
y2, X2 = dmatrices(data=mtcars, formula_like=lm_formula, return_type="dataframe")
lm4 = linear_model.OLS(endog=y2, exog=X2).fit()
lm4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     30.88
Date:                Wed, 09 Oct 2024   Prob (F-statistic):           5.05e-09
Time:                        21:02:44   Log-Likelihood:                -79.009
No. Observations:                  32   AIC:                             166.0
Df Residuals:                      28   BIC:                             171.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     34.1849      2.591     13.195      0.000      28.878      39.492
disp          -0.0188      0.010     -1.811      0.081      -0.040       0.002
cyl           -1.2274      0.797     -1.540      0.135      -2.861       0.406
hp            -0.0147      0.015     -1.002      0.325      -0.045       0.015
==============================================================================
Omnibus:                        2.942   Durbin-Watson:                   1.606
Prob(Omnibus):                  0.230   Jarque-Bera (JB):                2.558
Skew:                           0.675   Prob(JB):                        0.278
Kurtosis:                       2.692   Cond. No.                     1.51e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Linear model (part 5)

In [8]:
lm5 = LinearRegression().fit(X=X2, y=y2)
display(lm5.coef_)
display(lm5.feature_names_in_)
display(lm5.intercept_)

array([[ 0.        , -0.01883809, -1.22741994, -0.01467933]])

array(['Intercept', 'disp', 'cyl', 'hp'], dtype=object)

array([34.18491917])

In [9]:
X2[:3]

,Intercept,disp,cyl,hp
0,1.0,160.0,6.0,110.0
1,1.0,160.0,6.0,110.0
2,1.0,108.0,4.0,93.0


# Logistic regression (part 1)

In [10]:
lr1 = LogisticRegression(penalty=None).fit(X=X, y=mtcars["am"])
display(lr1.coef_)
display(lr1.feature_names_in_)
display(lr1.intercept_)

array([[-0.12488874,  0.84554446,  0.14442415]])

array(['disp', 'cyl', 'hp'], dtype=object)

array([-1.00603238])

# Logistic regression (part 2)

`Intercept = -1.0060`: The coefficients regresents log-odds, that is, the logarithmic representations of the odds ratios. Therefore, if you want to get the odds ratio, you perform an exponent of the coefficient. In the example below, the odds of a dependent variable being 1 is 36.6% holding all other variables constant (ie at zero)

In [11]:
math.exp(-1.0060)

0.36567877313053654

`disp = -0.1249`: This means that for every unit of increase in the displacement, the log-odds of the vehicle being manual (the dependent variable being 1), reduce by 0.1249. In other words, the odds of a car being manual reduce by 11.7%, from 100% to 88%

In [12]:
print(math.exp(-0.1249))

# the odds reduces frm 100% to 88%
print(1-(math.exp(-0.1249)))

0.8825851566874855
0.1174148433125145


`cyl = 0.8455`: For every unit increase of the number of cylinder, the log-odds of the car being manual increase by 0.8455, and the odds improve by 132%.

In [13]:
math.exp(0.8455)

2.3291420945232524

In [14]:
lr2 = sm.logit(formula="am ~ disp + cyl + hp", data=mtcars).fit()
lr2.summary()

Optimization terminated successfully.
         Current function value: 0.252684
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     am   No. Observations:                   32
Model:                          Logit   Df Residuals:                       28
Method:                           MLE   Df Model:                            3
Date:                Wed, 09 Oct 2024   Pseudo R-squ.:                  0.6259
Time:                        21:02:44   Log-Likelihood:                -8.0859
converged:                       True   LL-Null:                       -21.615
Covariance Type:            nonrobust   LLR p-value:                 5.725e-06
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.0060      3.735     -0.269      0.788      -8.327       6.315
disp          -0.1249      0.068     -1.824      0.068      -0.259       0.009
cyl            0.8455      1.210      0.699      0.485      -1.526       3.217
hp             0.1444      0.080      1.808      0.071      -0.012       0.301
==============================================================================

Possibly complete quasi-separation: A fraction 0.31 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [15]:
dir(lr2.summary())

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 '_repr_latex_',
 'add_extra_txt',
 'add_table_2cols',
 'add_table_params',
 'as_csv',
 'as_html',
 'as_latex',
 'as_text',
 'extra_txt',
 'tables']

In [16]:
results_as_html = lr2.summary().tables[1].as_html()
pd.read_html(StringIO(results_as_html), header=0, index_col=0)[0]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-1.0060,3.735,-0.269,0.788,-8.327,6.315
disp,-0.1249,0.068,-1.824,0.068,-0.259,0.009
cyl,0.8455,1.210,0.699,0.485,-1.526,3.217
hp,0.1444,0.080,1.808,0.071,-0.012,0.301


In [17]:
# Tells us if the means of 2 groups are statistically different/independent from each other

ttest_ind(
    x1=mtcars.mpg,
    x2=mtcars.disp,
    usevar="unequal" # consistent results with R
)

(-9.602360167373831, 7.978233909812945e-11, 31.146612273229884)

In [18]:
ztest(
    x1=mtcars.mpg,
    x2=mtcars.disp,
    usevar="unequal"
)

(-9.60236016737383, 7.813440988195989e-22)

In [20]:
# https://stackoverflow.com/questions/60291132/proportion-vs-contingency-chi-square-tests-giving-different-p-values-in-python
# You need a contingency table (frequency table)

smokers = pd.DataFrame({
    "Smokers": [83, 90, 129, 70],
    "Patients": [86, 93, 136, 82]
})

smokers

,Smokers,Patients
0,83,86
1,90,93
2,129,136
3,70,82


In [21]:
smokers.sum(axis = 1)

0    169
1    183
2    265
3    152
dtype: int64

In [22]:
# Tests if the frequencies of 2 populations/categorical variables are different from each other

proportions_chisquare(
    count=smokers.Smokers, # Same results with Patients
    nobs=smokers.sum(axis=1)
)

(0.42241146395430157,
 0.9355748751697771,
 (array([[ 83.,  86.],
         [ 90.,  93.],
         [129., 136.],
         [ 70.,  82.]]),
  array([[ 81.75292588,  87.24707412],
         [ 88.52535761,  94.47464239],
         [128.19245774, 136.80754226],
         [ 73.52925878,  78.47074122]])))

# Linear regression (Scipy)

statsmodel has better output. See this example output from scipy: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html#scipy.stats.linregress